## First order differential equation 
First order differential equations can be written in form as : $$ \large \frac{dy(t)}{dt} = f(y, t)$$
which describes rate of change of function y in respect to time. <br>

## Euler Method
because we have finite precision in computers we represent that equation in term of finite differences
$$\large \frac{\Delta y(t)}{\Delta t} = f(y, t)$$ <br>
$$\large \Delta y(t) = f(y, t) \Delta t$$ <br>
$$\large y^{n+1}(t) = y^{n} + f(y, t) \Delta t$$


In [2]:
%matplotlib inline
from ipywidgets import *
import numpy as np
import matplotlib.pyplot as plt

In [3]:
def radioactiveDecayAnalytical(t, dt, k):
    ts = np.arange(0, t + dt, dt)
    return ts, np.exp(-k * ts)

def radioactiveDecayDot(k, N):
    return -k * N

def radioactiveDecayEuler(t, dt, k):
    ts, Ns = [0], [1]
    N = 1
    for t in np.arange(dt, t + dt, dt):
        N = N + radioactiveDecayDot(k, N) * dt
        Ns.append(N)
        ts.append(t)
    return ts, Ns

def update(k = 1.0, t=0, dt=0.1):
    plt.figure(figsize=(12,8))
    ts, an = radioactiveDecayAnalytical(t, dt, k)
    plt.plot(ts, an, label='analytical,k={},dt={}'.format(k, dt))
    
    ts, euler = radioactiveDecayEuler(t, dt, k)
    plt.plot(ts, euler, label="euler,k={},dt={}".format(k, dt))
    plt.grid()
    plt.legend()
    
interact(update, k=FloatSlider(min=0, max=5, step=0.1, value=1.0) , 
         t=FloatSlider(min=0, max=15, step=0.1, value=5.0),
         dt=FloatSlider(min=0.01, max=2, step=0.01, value=0.2))

interactive(children=(FloatSlider(value=1.0, description='k', max=5.0), FloatSlider(value=5.0, description='t'…

<function __main__.update(k=1.0, t=0, dt=0.1)>

If you play long enough with simulation you can see that method isn't really precise ( even for such simple model as radiation decay ). 
* Euler method is first order accuracy,
* rather toy method then used in practice

## Leapfrog algoritm
Another simple method to evaluate such ordinary equation is leapfrog algorithm. It's two step algorithm ( second step is calculated with euler method ( with small enough step we can precise evaluate that value )). It's second order accuracy ( more precise than euler, but we have to evaluate second step ). <br>
$$ \large y^{n+1} = y^{n-1} + 2 \Delta t \cdot f(y^{n}, t^{n})$$

In [81]:
def eulerGetValueAtDt(dt, k, intervals):
    N = 1
    for t in np.linspace(0, dt, intervals):
        N = N + radioactiveDecayDot(k, N) * dt / intervals
    return N

def radioactiveDecayLeapfrog(t, dt, k, subintervals):
    ts, Ns = [0, dt], [1, eulerGetValueAtDt(dt, k, subintervals)]
    for t in np.arange(2 * dt, t + dt, dt):
        Ns.append(Ns[-2] + 2 * dt * radioactiveDecayDot(k, Ns[-1]))
        ts.append(t)
    
    return ts, Ns

def update(k = 1.0, t=0, dt=0.1, subintervals=1000):
    plt.figure(figsize=(12,8))
    ts, an = radioactiveDecayAnalytical(t, dt, k)
    plt.plot(ts, an, label='analytical,k={},dt={}'.format(k, dt))
    
    ts, euler = radioactiveDecayLeapfrog(t, dt, k, subintervals)
    plt.plot(ts, euler, label="leapfrog,k={},dt={},subintervals={}".format(k, dt, subintervals))
    plt.grid()
    plt.legend()
    
interact(update, k=FloatSlider(min=0, max=5, step=0.1, value=1.0) , 
         t=FloatSlider(min=0, max=15, step=0.1, value=5.0),
         dt=FloatSlider(min=0.01, max=2, step=0.01, value=0.2),
         subintervals=IntSlider(min=10, max=10000, step=10, value=500))

interactive(children=(FloatSlider(value=1.0, description='k', max=5.0), FloatSlider(value=5.0, description='t'…

<function __main__.update(k=1.0, t=0, dt=0.1, subintervals=1000)>

it's more accurate than euler, but then it lacks convergence. 

## mathematical pendulum Euler equation
applying euler method to more complex equation of mathematical pendulum

In [101]:
def pendulumOmegaDot(theta, g, r):
    return g / r * np.sin(theta)

def pendulumThetaDot(omega, theta):
    return omega

def pendulumEuler(t, dt, theta, omega, g, r):
    ts, omegas, thetas = [0], [omega], [theta * 180 / np.pi]
    for t in np.arange(dt, t + dt, dt):
        omega = omega + pendulumOmegaDot(theta, g, r)  * dt
        theta = theta + pendulumThetaDot(omega, theta) * dt
        omegas.append(omega)
        thetas.append(theta * 180 / np.pi)
        ts.append(t)
    return ts, thetas, omegas

def update(theta=0, omega=0, g=9.81, r=1, t=0, dt=0.1):
    plt.figure(figsize=(12,8))    
    
    ts, thetas, omegas = pendulumEuler(t, dt, theta * np.pi / 180, omega, g, r)
    plt.plot(ts, thetas, label="euler,g={},r={},dt={}".format(g, r, dt))
   
    plt.xlabel("time [t]")
    plt.ylabel("angle [theta]")
    plt.grid()
    plt.legend()
    
interact(update, theta=IntSlider(min=0, max=360, step=1, value=180) , 
                 omega=FloatSlider(min=-10, max=10, step=0.01, value=0),
                 g=FloatSlider(min=1, max=20, step=0.01, value=9.81),
                 r=FloatSlider(min=0.1, max=10, step=0.05, value=1),
                 t=FloatSlider(min=0.1, max=10, step=0.1, value=5),
                 dt=FloatSlider(min=0.01, max=2, step=0.01, value=0.05))

interactive(children=(IntSlider(value=180, description='theta', max=360), FloatSlider(value=0.0, description='…

<function __main__.update(theta=0, omega=0, g=9.81, r=1, t=0, dt=0.1)>

Plotting energies of mathematical pendulum ( potential, kinetic and total energy) ( as you can see total energy isn't equal at all time, ( it oscillates) because of accuracy of our method) 

In [105]:
def pendulumOmegaDot(theta, g, r):
    return g / r * np.sin(theta)

def pendulumThetaDot(omega, theta):
    return omega

def pendulumEulerEnergy(t, dt, theta, omega, g, r, m):
    ts, Ek, Ep = [], [], []
    for t in np.arange(0, t + dt, dt):
        omega = omega + pendulumOmegaDot(theta, g, r)  * dt
        theta = theta + pendulumThetaDot(omega, theta) * dt
        Ek.append( (omega* r)**2 * m / 2 )
        Ep.append(m * g * np.cos(theta) * r)
        ts.append(t)
    return ts, Ek, Ep

def update(theta=0, omega=0, g=9.81, r=1, t=0, dt=0.1, m=1):
    plt.figure(figsize=(12,8))    
    
    ts, Ek, Ep = pendulumEulerEnergy(t, dt, theta * np.pi / 180, omega, g, r, m)
    plt.plot(ts, Ek, label="Kinetic Energy,euler,g={},r={},dt={}".format(g, r, dt))
    plt.plot(ts, Ep, label="Potential Energy,euler,g={},r={},dt={}".format(g, r, dt))
    plt.plot(ts, np.array(Ek) + np.array(Ep), label="Total Energy")
    plt.xlabel("time [t]")
    plt.ylabel("Energy ")
    plt.grid()
    plt.legend()
    
interact(update, theta=IntSlider(min=0, max=360, step=1, value=180) , 
                 omega=FloatSlider(min=-10, max=10, step=0.01, value=0),
                 g=FloatSlider(min=1, max=20, step=0.01, value=9.81),
                 r=FloatSlider(min=0.1, max=10, step=0.05, value=1),
                 t=FloatSlider(min=0.1, max=10, step=0.1, value=5),
                 dt=FloatSlider(min=0.01, max=2, step=0.01, value=0.05),
                 m=FloatSlider(min=0.1, max=10, step=0.1, value=0.1))

interactive(children=(IntSlider(value=180, description='theta', max=360), FloatSlider(value=0.0, description='…

<function __main__.update(theta=0, omega=0, g=9.81, r=1, t=0, dt=0.1, m=1)>

# Lisajous Figures
Lisajous figures are interesting case of simple equation that can create really interesting images. Those figures are described as :
$$\large x = sin(at)$$
$$\large y = cos(bt + \alpha)$$

In [109]:
def update(t, a, b, dt, shift):
    plt.figure(figsize=(12,12))    
    
    shift = shift * np.pi / 180
    ts, a1, a2 = [], [], []
    for t in np.arange(0, t + dt, dt):
        a1.append(np.sin(a * t))
        a2.append(np.cos(b * t + shift))
    plt.plot(a1, a2, label="a={},b={},shift={},dt={}".format(a,b,shift,dt))
    plt.xlabel("angle1")
    plt.ylabel("angle2")
    plt.legend()
    
interact(update, t=FloatSlider(min=0, max=100, step=1,       value=50),
                 a=FloatSlider(min=-10, max=10, step=0.01,   value=-4.6),
                 b=FloatSlider(min=-10, max=10, step=0.01,   value=5),
                 dt=FloatSlider(min=0.01, max=0.5, step=0.01, value=0.01),
                 shift=IntSlider(min=0, max=360, value=0))

interactive(children=(FloatSlider(value=50.0, description='t', step=1.0), FloatSlider(value=-4.6, description=…

<function __main__.update(t, a, b, dt, shift)>

In [132]:
# Harmonic oscillator derivative of velocity
def harmonicOscVelocityDot(g, m, k, x):
    return g - k /m * x

# derivative of position
def harmonicOscPositionDot(v, x):
    return v

def harmonicOscillatorEuler(t, dt, g, k, m, x, v):
    ts, vs, xs = [0], [v], [x]
    for t in np.arange(dt, t + dt, dt):
        v = v + harmonicOscVelocityDot(g, m, k, x)  * dt
        x = x + harmonicOscPositionDot(v, x) * dt
        vs.append(v)
        xs.append(x)
        ts.append(t)
    return ts, vs, xs

def update(t, g, k, dt, m, x, v):
    plt.figure(figsize=(12,12))    
    
    ts, xs,vs = harmonicOscillatorEuler(t, dt, g, k, m, x, v)
    plt.plot(ts, xs,label="g={},k={},dt={},m={}".format(g,k,dt,m))
    plt.legend()
    plt.grid()
    plt.xlabel("time")
    plt.ylabel("position")
    
interact(update, t=FloatSlider(min=0, max=100, step=1,       value=10),
                 dt=FloatSlider(min=0.01, max=0.5, step=0.01, value=0.01),
                 g=FloatSlider(min=0, max=20, step=0.01,   value=9.81),
                 k=FloatSlider(min=0.01, max=10, step=0.01,   value=1),
                 m=FloatSlider(min=0.1, max=10, step=0.1, value=1),
                 x=FloatSlider(min=-10, max=10, step=0.1, value=0),
                 v=FloatSlider(min=-10, max=10, step=0.1, value=0))

interactive(children=(FloatSlider(value=10.0, description='t', step=1.0), FloatSlider(value=9.81, description=…

<function __main__.update(t, g, k, dt, m, x, v)>

## more accurate methods

## Midpoint
another method of second order accuracy is MidPoint. This method can be formulated as:
$$ \large k_{1} = h \cdot f(y^{n}, t^{n})$$<br>
$$ \large k_{2} = h \cdot f(y^{n} + 0.5 k_{1}, t^{n} + 0.5 h)$$<br>
$$ \large y^{n+1} = y^{n} + k_{2}$$<br>

In [135]:
def radioactiveDecayMidpoint(t, dt, k):
    ts, Ns = [0], [1]
    N = 1
    for t in np.arange(dt, t + dt, dt):
        k1 = dt * radioactiveDecayDot(k, N) 
        k2 = dt * radioactiveDecayDot(k, N + 0.5 * k1)
        N = N + k2
        Ns.append(N)
        ts.append(t)
    return ts, Ns

def update(k = 1.0, t=0, dt=0.1):
    plt.figure(figsize=(12,8))
    ts, an = radioactiveDecayAnalytical(t, dt, k)
    plt.plot(ts, an, label='analytical,k={},dt={}'.format(k, dt))
    
    ts, euler = radioactiveDecayMidpoint(t, dt, k)
    plt.plot(ts, euler, label="Midpoint,k={},dt={}".format(k, dt))
    plt.grid()
    plt.legend()
    
interact(update, k=FloatSlider(min=0, max=5, step=0.1, value=1.0) , 
         t=FloatSlider(min=0, max=15, step=0.1, value=5.0),
         dt=FloatSlider(min=0.01, max=2, step=0.01, value=0.2))

interactive(children=(FloatSlider(value=1.0, description='k', max=5.0), FloatSlider(value=5.0, description='t'…

<function __main__.update(k=1.0, t=0, dt=0.1)>

Midpoint results in much better results than euler and doesn't have problem with convergence as leapfrog ( it has the same order of accuracy as leapfrog).
With not really that much effort ( adding one-two extra line of code ) we can easily get that results 

## Runge-Kutta IV
This is 4 order of accuracy method. It can be used in many applications where accuracy matters. The implementation isn't as that complicated as others ( which is great because we achieve great accuracy in comparasion with other).

$$ \large k_{1} = h \cdot f(y^{n},t^{n})$$ <br>
$$ \large k_{2} = h \cdot f(y^{n} + 0.5 k_{1},t^{n} + 0.5 h)$$ <br>
$$ \large k_{3} = h \cdot f(y^{n} + 0.5 k_{2},t^{n} + 0.5 h)$$ <br>
$$ \large k_{4} = h \cdot f(y^{n} + k_{3},t^{n} + h)$$ <br>
$$ \large y^{n+1} = y^{n} + \frac{1}{6}(k_{1} + 2k_{2} + 2k_{3} + k_{4}) $$ <br> 

In [137]:
def radioactiveDecayRKIV(t, dt, k):
    ts, Ns = [0], [1]
    N = 1
    for t in np.arange(dt, t + dt, dt):
        k1 = dt * radioactiveDecayDot(k, N) 
        k2 = dt * radioactiveDecayDot(k, N + 0.5 * k1)
        k3 = dt * radioactiveDecayDot(k, N + 0.5 * k2)
        k4 = dt * radioactiveDecayDot(k, N + k3)
        N = N + 1/6 * (k1 + 2 * k2 + 2 * k3 + k4)
        Ns.append(N)
        ts.append(t)
    return ts, Ns

def update(k = 1.0, t=0, dt=0.1):
    plt.figure(figsize=(12,8))
    ts, an = radioactiveDecayAnalytical(t, dt, k)
    plt.plot(ts, an, label='analytical,k={},dt={}'.format(k, dt))
    
    ts, euler = radioactiveDecayRKIV(t, dt, k)
    plt.plot(ts, euler, label="Runge Kutta IV,k={},dt={}".format(k, dt))
    plt.grid()
    plt.legend()
    
interact(update, k=FloatSlider(min=0, max=5, step=0.1, value=1.0) , 
         t=FloatSlider(min=0, max=15, step=0.1, value=5.0),
         dt=FloatSlider(min=0.01, max=2, step=0.01, value=1))

interactive(children=(FloatSlider(value=1.0, description='k', max=5.0), FloatSlider(value=5.0, description='t'…

<function __main__.update(k=1.0, t=0, dt=0.1)>